In [2]:
import arxiv
import urllib.request as libreq
import feedparser
import pdfminer.layout
import pdfminer.high_level
from io import StringIO
from pdfminer.layout import LAParams
from bs4 import BeautifulSoup as bs
import nltk
import re
import heapq
import boto3

## Read PDF

In [3]:
text=pdfminer.high_level.extract_text('C:\\Users\\Al\\Documents\\ByteSizeArxiv\\library/9905014v1.pdf', codec='utf-8', laparams=None)

In [4]:
text

'9\n9\n9\n1\n \ny\na\nM\n1\n2\n \n \n]\n\n \n\nG\nL\n.\ns\nc\n[\n \n \n1\nv\n4\n1\n0\n5\n0\n9\n9\n/\ns\nc\n:\nv\ni\nX\nr\na\n\nHierarchical Reinforcement Learning with the MAXQ Value\nFunction Decomposition\n\nThomas G. Dietterich\nDepartment of Computer Science\nOregon State University\nCorvallis, OR 97331\ntgd@cs.orst.edu\n\nFebruary 1, 2008\n\nAbstract\n\nThis paper presents a new approach to hierarchical reinforcement learning based on decomposing\nthe target Markov decision process (MDP) into a hierarchy of smaller MDPs and decomposing\nthe value function of the target MDP into an additive combination of the value functions of the\nsmaller MDPs. The decomposition, known as the MAXQ decomposition, has both a procedural\nsemantics—as a subroutine hierarchy—and a declarative semantics—as a representation of the\nvalue function of a hierarchical policy. MAXQ uniﬁes and extends previous work on hierar-\nchical reinforcement learning by Singh, Kaelbling, and Dayan and Hinton. It is base

## Clean PDF

In [5]:
cleanedText = text 

In [6]:
cleanedText = re.sub(r'-\n','-', cleanedText) #Remove -\n issues
cleanedText = re.sub(r'\n',' ',cleanedText)#Get rid of new lines replace with spaces

cleanedText = re.sub(r'- ','',cleanedText) #To get rid of hyphens next lines

cleanedText = re.sub(r'\x0c','', cleanedText) #Remove page breaks

In [7]:
cleanedText = cleanedText.split("Abstract ",1)[1]

In [8]:
cleanedText

' This paper presents a new approach to hierarchical reinforcement learning based on decomposing the target Markov decision process (MDP) into a hierarchy of smaller MDPs and decomposing the value function of the target MDP into an additive combination of the value functions of the smaller MDPs. The decomposition, known as the MAXQ decomposition, has both a procedural semantics—as a subroutine hierarchy—and a declarative semantics—as a representation of the value function of a hierarchical policy. MAXQ uniﬁes and extends previous work on hierar-chical reinforcement learning by Singh, Kaelbling, and Dayan and Hinton. It is based on the assumption that the programmer can identify useful subgoals and deﬁne subtasks that achieve these subgoals. By deﬁning such subgoals, the programmer constrains the set of policies that need to be considered during reinforcement learning. The MAXQ value function decomposition can represent the value function of any policy that is consistent with the given 

## Tokenize and Calculate Freq

In [9]:
sentence_tokens= nltk.sent_tokenize(cleanedText)
word_tokens = nltk.word_tokenize(cleanedText)

In [17]:
#Calculate frequency
stopwords = nltk.corpus.stopwords.words('english')
wordFreqs = {}

for word in word_tokens:
    if word not in stopwords:
        if word not in wordFreqs.keys():
            wordFreqs[word] = 1
        else:
            wordFreqs[word] +=1

In [11]:
wordFreqs

{'This': 114,
 'paper': 37,
 'presents': 5,
 'new': 16,
 'approach': 25,
 'hierarchical': 117,
 'reinforcement': 30,
 'learning': 160,
 'based': 23,
 'decomposing': 2,
 'target': 11,
 'Markov': 12,
 'decision': 8,
 'process': 9,
 '(': 1338,
 'MDP': 50,
 ')': 1335,
 'hierarchy': 58,
 'smaller': 2,
 'MDPs': 6,
 'value': 183,
 'function': 203,
 'additive': 1,
 'combination': 2,
 'functions': 45,
 '.': 1496,
 'The': 230,
 'decomposition': 43,
 ',': 2563,
 'known': 5,
 'MAXQ': 214,
 'procedural': 2,
 'semantics—as': 2,
 'subroutine': 25,
 'hierarchy—and': 1,
 'declarative': 2,
 'representation': 49,
 'policy': 334,
 'uniﬁes': 1,
 'extends': 1,
 'previous': 8,
 'work': 17,
 'hierar-chical': 2,
 'Singh': 21,
 'Kaelbling': 24,
 'Dayan': 8,
 'Hinton': 6,
 'It': 45,
 'assumption': 6,
 'programmer': 24,
 'identify': 4,
 'useful': 7,
 'subgoals': 3,
 'deﬁne': 23,
 'subtasks': 58,
 'achieve': 12,
 'By': 13,
 'deﬁning': 11,
 'constrains': 4,
 'set': 49,
 'policies': 60,
 'need': 26,
 'considered': 4

In [12]:
mostFreqy = max(wordFreqs.values())

for word in wordFreqs.keys():
    wordFreqs[word] = (wordFreqs[word]/mostFreqy)
    
print(wordFreqs)

{'This': 0.0444791260241904, 'paper': 0.01443620756925478, 'presents': 0.0019508388607101053, 'new': 0.006242684354272337, 'approach': 0.009754194303550527, 'hierarchical': 0.04564962934061646, 'reinforcement': 0.011705033164260631, 'learning': 0.06242684354272337, 'based': 0.008973858759266484, 'decomposing': 0.0007803355442840422, 'target': 0.004291845493562232, 'Markov': 0.0046820132657042525, 'decision': 0.0031213421771361686, 'process': 0.00351150994927819, '(': 0.5220444791260241, 'MDP': 0.019508388607101055, ')': 0.5208739758095982, 'hierarchy': 0.02262973078423722, 'smaller': 0.0007803355442840422, 'MDPs': 0.0023410066328521262, 'value': 0.07140070230198986, 'function': 0.07920405774483028, 'additive': 0.0003901677721420211, 'combination': 0.0007803355442840422, 'functions': 0.017557549746390948, '.': 0.5836909871244635, 'The': 0.08973858759266484, 'decomposition': 0.016777214202106906, ',': 1.0, 'known': 0.0019508388607101053, 'MAXQ': 0.08349590323839251, 'procedural': 0.00078

## Sentance Scores

In [13]:
sentence_scores = {}

for sentence in sentence_tokens:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in wordFreqs.keys():
            if len(sentence.split(' ')) <30:
                if sentence not in sentence_scores.keys():
                    sentence_scores[sentence] = wordFreqs[word]
                else:
                    sentence_scores[sentence] += wordFreqs[word]

In [14]:
sentence_scores

{'The decomposition, known as the MAXQ decomposition, has both a procedural semantics—as a subroutine hierarchy—and a declarative semantics—as a representation of the value function of a hierarchical policy.': 2.9781506047600477,
 'MAXQ uniﬁes and extends previous work on hierar-chical reinforcement learning by Singh, Kaelbling, and Dayan and Hinton.': 2.669137729223566,
 'It is based on the assumption that the programmer can identify useful subgoals and deﬁne subtasks that achieve these subgoals.': 0.647288333983613,
 'By deﬁning such subgoals, the programmer constrains the set of policies that need to be considered during reinforcement learning.': 1.728443230589153,
 'The MAXQ value function decomposition can represent the value function of any policy that is consistent with the given hierarchy.': 1.0819352321498246,
 'The decomposition also creates opportunities to exploit state abstractions, so that individual MDPs within the hierarchy can ignore large parts of the state space.': 1

In [15]:
summaryML = heapq.nlargest(10, sentence_scores, key = sentence_scores.get)

In [16]:
summaryML

['.',
 'Nonetheless, it draws a sample according to P (s′|x, y, a), receives a reward R(s′|x, y, a), and updates its estimate of V (a, x) (line 5 of MAXQ-Q).',
 'Then the four possible subroutines are MSniﬀ(East, N orth), MSniﬀ(East, South), MBack(East, N orth), and MBack(East, South).',
 's′ P (s′|s, a)[R(s′|s, a) + γV π(0, s)], where V π(0, s) is the value Corollary 3 Let πg(s) = argmaxa function computed by the MAXQ hierarchy.',
 '+ C π(a1, s, a2) + C π(0, s, a1),  (12)  where a0, a1, .',
 'Hauskrecht, M., Meuleau, N., Boutilier, C., Kaelbling, L., & Dean, T. (1998).',
 'Hence, the right-hand sides of (21) have the same value, and we can conclude that  C π(i, s1, j) = C π(i, s2, j).',
 'At each level i, we compute values for C π(i, s, π(s)) (or V π(i, s), if i is primitive) according to the decomposition equations.',
 'For example, suppose that the two best actions at state s are a1 and a2, that Q(s, a1) = Q(s, a2), and that ω(a1, a2).',
 '• C(GotoGoalLmk(gl), s, M axGotoLmk(l)) the